In [9]:
import boto3
import pandas as pd
import json
from io import StringIO

s3 = boto3.client('s3')
bucket = 'spotify-genius-featurestore'  # replace with your input bucket name
output_bucket = 'spotify-genius-featurestore'  # replace with your output bucket name
output_key = 'spotify-reco-db/actual-database.csv'  # replace with your desired output file name

# List all objects in the bucket
objects = s3.list_objects_v2(Bucket=bucket, Prefix='backup-processed-features/')['Contents'][1:]

# Initialize an empty DataFrame
# print(objects)
df = pd.DataFrame()
cols = 'danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,duration_ms,popularity,explicit,embedding_1,embedding_2,embedding_3,embedding_4,embedding_5,embedding_6,embedding_7,embedding_8,embedding_9,embedding_10,embedding_11,embedding_12,embedding_13,embedding_14,embedding_15,embedding_16,embedding_17,embedding_18,embedding_19,embedding_20,embedding_21,embedding_22,embedding_23,embedding_24,embedding_25'.split(',')
        
    
def delete_s3_object(bucket_name, object_key):
    response = s3.delete_objects(
        Bucket=bucket_name,
        Delete={
            'Objects': [
                {
                    'Key': object_key
                },
            ]
        }
    )
    return response

# Use the function
# print(objects)
# Process each JSON file
for obj in objects:
    key = obj['Key']
    json_obj = s3.get_object(Bucket=bucket, Key=key)
    json_data = json_obj['Body'].read().decode('utf-8')
    json_dict = json.loads(json_data)  # load the JSON data as a Python dictionary
    
    # Selectively normalize the JSON data
    keys_to_normalize = ['song_id','name', 'artist', 'features']  # replace with your keys
    normalized_data = {key: json_dict[key] for key in keys_to_normalize if key in json_dict}
    json_df = pd.json_normalize(normalized_data)  # flatten the selected JSON data into a DataFrame
    json_df2 = pd.DataFrame(json_df["features"].to_list(), columns=cols)
    json_df2.insert(0, 'song_id', json_df['song_id'])
    json_df2.insert(1, 'name', json_df['name'])
    json_df2.insert(2, 'artist', json_df['artist'])
    # print(json_df2)
    df = pd.concat([df, json_df2])  # concatenate the DataFrames
    delete_s3_object(bucket, key)

    
df.head(15)


,song_id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,...,embedding_16,embedding_17,embedding_18,embedding_19,embedding_20,embedding_21,embedding_22,embedding_23,embedding_24,embedding_25
0,01K4zKU104LyJ8gMb7227B,Nothing New (feat. Phoebe Bridgers) (Taylor’s ...,Taylor Swift,0.606,0.377,0,-9.455,1,0.0275,0.81700,...,-0.056408,18.093826,21.265907,9.484246,9.039261,2.669970,-12.481927,1.587471,-10.933788,-8.480564
0,02M6vucOvmRfMxTXDUwRXu,7/11,Beyoncé,0.747,0.705,9,-5.137,0,0.1260,0.01280,...,33.395554,3.669261,4.682726,14.148369,-18.077756,-10.910452,3.769870,-3.548786,-11.688713,-10.942340
0,0314PeD1sQNonfVWix3B2K,ENERGY (feat. Beam),Beyoncé,0.903,0.519,1,-9.151,1,0.2600,0.28100,...,9.518229,17.530536,-13.720719,-3.086728,-15.947706,-6.070911,3.914216,-1.247357,-15.722782,-8.735804
0,0A1JLUlkZkp2EFrosoNQi0,Labyrinth,Taylor Swift,0.406,0.306,0,-15.480,1,0.0517,0.78500,...,8.764783,16.703382,8.249090,6.479297,1.257009,-27.556654,1.475680,-13.152661,6.472265,-4.756790
0,0BiqmkasE5FdrChwKfVp8X,Bigger Than The Whole Sky,Taylor Swift,0.422,0.235,6,-12.379,1,0.0568,0.82600,...,18.277492,8.701950,-9.247032,1.107909,-11.319336,-15.594052,-0.360566,-14.622891,-16.579120,3.195654
0,0E6PsO3ymCfUh7pJQjBgkj,APESHIT,The Carters,0.705,0.784,2,-6.477,1,0.2710,0.01330,...,13.802790,4.465309,-5.508799,5.474669,-9.597346,-6.151166,9.653696,3.445707,2.000906,-3.690360
0,0Fl4eWzVaMUOdXcOrj6F1q,VIRGO'S GROOVE,Beyoncé,0.683,0.850,11,-5.042,0,0.0699,0.07810,...,10.479103,11.236016,16.269305,-1.404986,-2.012585,-31.885934,6.019898,-12.582999,10.539216,-2.882682
0,0GLUBbX4daHJkT3RQHEOia,Upgrade U (feat. Jay-Z),Beyoncé,0.593,0.680,7,-5.556,1,0.3350,0.00182,...,26.752002,-6.934932,-29.401395,10.179193,-20.530202,23.534079,6.352517,-6.491036,9.309036,13.102896
0,0GzmMQizDeA2NVMUaZksv0,Mi Gente (feat. Beyoncé),J Balvin,0.759,0.716,11,-6.360,0,0.0818,0.02230,...,4.302624,4.380778,-15.040108,-15.525858,4.960939,0.246724,-4.651271,6.256678,-2.730536,-5.190676
0,0Jlcvv8IykzHaSmj49uNW8,the 1,Taylor Swift,0.777,0.357,0,-6.942,1,0.0522,0.75700,...,25.739292,9.818881,-17.613062,-12.776891,2.307610,-3.711609,6.068336,-22.429742,-0.431599,14.499729


In [10]:
existing_csv_obj = s3.get_object(Bucket=bucket, Key=output_key)
existing_csv_data = existing_csv_obj['Body'].read().decode('utf-8')
existing_df = pd.read_csv(StringIO(existing_csv_data),)
print(existing_df)
append_df = pd.DataFrame()
append_df = pd.concat([existing_df, df])
# Write the DataFrame to a CSV file
# df.head(15)
append_df.head(15)


                    song_id                                  name  \
0    01u6AEzGbGbQyYVdxajxqk                        Don't Be Cruel   
1    06HM8m5kmKG8AOt7eelGhI                       Put A Lid On It   
2    086Oiv5FPVmHkzmLYvfsol                          Best Believe   
3    08mG3Y1vljYA6bvDt4Wqkj                         Back In Black   
4    0Afai8u1STCaDyTTm1gEZf                     Shake Some Action   
..                      ...                                   ...   
258  6otiaV2fagE3s8IvP6WkwG                        I'm Just a Kid   
259  7j31rVgGX9Q2blT92VBEA0                             Teenagers   
260  7kDUspsoYfLkWnZR7qwHZl  my ex's best friend (with blackbear)   
261  7lRlq939cDG4SzWOF4VAnd              I'm Not Okay (I Promise)   
262  7w0gu9n8vp3sE0xZFCxcFu                      Drown In My Mind   

                   artist  danceability  energy  key  loudness  mode  \
0           Elvis Presley         0.697   0.550    2   -11.496     1   
1    Squirrel Nut Zippers  

,song_id,name,artist,danceability,energy,key,loudness,mode,speechiness,acousticness,...,embedding_16,embedding_17,embedding_18,embedding_19,embedding_20,embedding_21,embedding_22,embedding_23,embedding_24,embedding_25
0,01u6AEzGbGbQyYVdxajxqk,Don't Be Cruel,Elvis Presley,0.697,0.550,2,-11.496,1,0.1790,0.8560,...,22.763881,-10.222959,-19.197018,-15.008783,-14.105286,-20.047570,5.468243,-10.577241,-4.244537,-6.483427
1,06HM8m5kmKG8AOt7eelGhI,Put A Lid On It,Squirrel Nut Zippers,0.496,0.547,0,-8.605,0,0.1200,0.6210,...,-10.026583,18.530731,3.936493,-3.831682,-24.870016,15.857049,18.546781,25.715571,-2.556938,-1.540036
2,086Oiv5FPVmHkzmLYvfsol,Best Believe,The Dip,0.726,0.799,0,-7.092,1,0.0304,0.1700,...,4.319392,20.119860,-0.662674,-19.442136,13.817505,-6.824602,-7.718642,-16.872597,-3.630400,-17.555923
3,08mG3Y1vljYA6bvDt4Wqkj,Back In Black,AC/DC,0.310,0.700,9,-5.678,1,0.0470,0.0110,...,17.267427,-13.863575,-9.675647,6.918551,-6.678916,-12.873291,0.238466,-2.322251,-4.363672,-18.518605
4,0Afai8u1STCaDyTTm1gEZf,Shake Some Action,Flamin' Groovies,0.495,0.632,8,-12.941,1,0.0404,0.0254,...,-2.088394,8.189940,12.534375,-6.282429,-3.236247,5.649349,0.926731,-5.281596,-12.879724,-12.828225
5,0BlhdiMCG3coa1l0q03WLp,As Good As You've Been to This World,Janis Joplin,0.650,0.740,0,-9.026,0,0.0458,0.2820,...,17.167653,14.684744,10.270924,3.284428,7.417143,0.177462,25.957083,-0.766999,-6.739751,-0.440036
6,0CsM8VGDi38kusMv3pxyj1,True - Single Edit,Spandau Ballet,0.742,0.533,0,-9.098,1,0.0288,0.4590,...,-4.950219,-1.130465,5.662639,0.976261,-9.142268,-0.005979,7.032355,22.084909,14.452334,-6.395622
7,0F6NRy1dhuXiRdD3VmuZVD,Chops,Argonaut & Wasp,0.790,0.684,7,-5.401,0,0.0396,0.0402,...,-2.072310,0.245946,-12.776121,-12.486414,-2.262965,2.655146,7.385602,17.609778,-1.645570,5.847911
8,0JtkhMvhKOAeKGCQmau1LF,Late Last Night,Kowloon,0.865,0.447,1,-10.920,1,0.0426,0.3390,...,13.831303,13.128052,-0.153366,7.184338,-8.951353,0.961860,-0.440657,13.327148,24.000788,12.659639
9,0NZjt6ZlCVIZarakXF2WkQ,She Gave Me the Keys,The Dip,0.610,0.557,11,-7.682,1,0.0363,0.7310,...,4.877071,6.892778,1.943206,-4.797184,-18.343231,11.668515,8.103193,1.086180,15.061832,0.936393


In [11]:
csv_buffer = StringIO()
append_df.to_csv(csv_buffer, index=False)
s3.put_object(Bucket=bucket, Key=output_key, Body=csv_buffer.getvalue())

{'ResponseMetadata': {'RequestId': '7TJ0PV9QXJZKQX7H',
  'HostId': 'FI2EeBgHCjlrMKLcNrhGburvt1KNVY9GP+ono+LpcMJXweDelA5Lklke0Ly8EWRlXcZ5IfKy57E=',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amz-id-2': 'FI2EeBgHCjlrMKLcNrhGburvt1KNVY9GP+ono+LpcMJXweDelA5Lklke0Ly8EWRlXcZ5IfKy57E=',
   'x-amz-request-id': '7TJ0PV9QXJZKQX7H',
   'date': 'Fri, 23 Jun 2023 05:34:27 GMT',
   'x-amz-server-side-encryption': 'AES256',
   'etag': '"c27577b67ba6f5062834137ff3946a22"',
   'server': 'AmazonS3',
   'content-length': '0'},
  'RetryAttempts': 0},
 'ETag': '"c27577b67ba6f5062834137ff3946a22"',
 'ServerSideEncryption': 'AES256'}